In [2]:
import numpy as np

In [3]:
cutoff = 4
N = 4
potential = 'QHO'

In [4]:
q_coeff = np.sqrt(1/(2*N))
q = q_coeff * np.diag(np.linspace(-N + 1, N - 1, N))

In [5]:
q

array([[-1.06066017,  0.        ,  0.        ,  0.        ],
       [ 0.        , -0.35355339,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.35355339,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  1.06066017]])

In [8]:
p2_coeff = N/2

diagonal = np.full(N, 2)
off_diagonal = np.full(N - 1, -1)
matrix = np.diag(diagonal) + np.diag(off_diagonal, k=1) + np.diag(off_diagonal, k=-1)

p2 = p2_coeff*matrix

In [9]:
p2

array([[ 4., -2.,  0.,  0.],
       [-2.,  4., -2.,  0.],
       [ 0., -2.,  4., -2.],
       [ 0.,  0., -2.,  4.]])

In [ ]:
#b_cutoff = cutoff

# Calculate q^2 and q^3
q2 = np.matmul(q, q)
q3 = np.matmul(q2, q)

# Bosonic and fermionic identities
#I_b = np.eye(b_cutoff)
I_f = np.eye(2)

# Superpotential derivatives
if potential == 'QHO':
    W_prime = q  # W'(q) = q
    W_double_prime = I_b  # W''(q) = 1

elif potential == 'AHO':
    W_prime = q + q3  # W'(q) = q + q^3
    W_double_prime = I_b + 3 * q2  # W''(q) = 1 + 3q^2

elif potential == 'DW':
    W_prime = q + q2 + I_b  # W'(q) = q + q^2 + 1
    W_double_prime = I_b + 2 * q  # W''(q) = 1 + 2q

else:
    raise ValueError("Not a valid potential")

# Kinetic term: p^2 (finite difference Laplacian)
#p2 = np.matmul(p, p)

# Commutator term [b^†, b] = -Z
Z = np.array([[1, 0], [0, -1]])  # Pauli Z matrix for fermion number
commutator_term = np.kron(Z, W_double_prime)

# Kinetic term (bosonic and fermionic parts)
kinetic_term = np.kron(I_f, p2)

# Potential term (W' contribution)
potential_term = np.kron(I_f, np.matmul(W_prime, W_prime))

# Construct the full Hamiltonian
H_SQM = 0.5 * (kinetic_term + potential_term + commutator_term)
H_SQM[np.abs(H_SQM) < 1e-12] = 0  # Eliminate small numerical artifacts


In [11]:
np.sort(np.linalg.eig(H_SQM)[0])

array([0.0625    , 1.0625    , 1.28172359, 2.28172359, 2.5625    ,
       3.34327641, 3.5625    , 4.34327641])

In [12]:
min(np.sort(np.linalg.eig(H_SQM)[0]))

np.float64(0.06250000000000022)